In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import csv

In [2]:
# header = ['itn','x(1)','norm r','norm Ar','compatible','LS','norm A','cond A']

# mol = 'heh+'
# basis = '6-31g'

# mol = 'heh+'
# basis = '6-311G'

# mol = 'lih'
# basis = '6-31g'

mol = 'c2h4'
basis = 'sto-3g'

ltflag = '_lowtol_'

newmeth = {'PG':'Tied',
           'PGR':'TiedE',
           'HB':'Herm',
           'HBR':'HermE',
           'MO':'Symm',
           'MOR':'SymmE'}

In [3]:
methods = ['PG','HB','MO']
rf = ['','R']
logfiles = []
fullmethods = []
fulllabels = []
for meth in methods:
    for rflag in rf:
        if meth == 'MO':
            logfiles.append('./logs'+meth+'iter/'+mol+rflag+'_'+basis+'_fit2.out')
        else:
            logfiles.append('./logs'+meth+'/'+mol+rflag+ltflag+basis+'_fit2.out')
        fullmethods.append(meth)
        fulllabels.append(newmeth[meth+rflag])
    
print(logfiles)
print(fullmethods)

['./logsPG/c2h4_lowtol_sto-3g_fit2.out', './logsPG/c2h4R_lowtol_sto-3g_fit2.out', './logsHB/c2h4_lowtol_sto-3g_fit2.out', './logsHB/c2h4R_lowtol_sto-3g_fit2.out', './logsMOiter/c2h4_sto-3g_fit2.out', './logsMOiter/c2h4R_sto-3g_fit2.out']
['PG', 'PG', 'HB', 'HB', 'MO', 'MO']


In [4]:
def grabnorm(fname, meth):
    i = 0
    itr = []
    normr = []
    with open(fname, "r") as csv_file:
        print(fname)
        csv_reader = csv.reader(csv_file, delimiter=' ')
        done = False
        while not done:
            skip = next(csv_reader)
            if len(skip) >= 4:
                if skip[3] == 'itn':
                    break
        for lines in csv_reader:
            i += 1
            if i % 22 == 0:
                lines = next(csv_reader)
                if lines[0]=="LSMR" and lines[1]=="finished":
                    break
                lines = next(csv_reader)
                if lines[0]=="LSMR" and lines[1]=="finished":
                    break
                i += 2
            if len(lines)==2:
                lines = next(csv_reader)
                if lines[0]=="LSMR" and lines[1]=="finished":
                    break
            goodcols = []
            for j in range(len(lines)):
                if lines[j] != '':
                    goodcols.append(j)
            itr.append( int(lines[goodcols[0]]) )
            normr.append( float(lines[goodcols[2]])**2 )
    return itr, normr

In [5]:
itrs = []
normrs = []
for j in range(len(fullmethods)):
    itr, normr = grabnorm(logfiles[j],fullmethods[j])
    itrs.append(itr)
    normrs.append(normr)

./logsPG/c2h4_lowtol_sto-3g_fit2.out
./logsPG/c2h4R_lowtol_sto-3g_fit2.out
./logsHB/c2h4_lowtol_sto-3g_fit2.out
./logsHB/c2h4R_lowtol_sto-3g_fit2.out
./logsMOiter/c2h4_sto-3g_fit2.out
./logsMOiter/c2h4R_sto-3g_fit2.out


In [6]:
matplotlib.rcParams.update({'font.size': 24})
plt.figure(figsize=(8,7))

plt.ticklabel_format(axis='both', style='sci', scilimits=(3,3))

for j in range(len(itrs)):
    if j % 2 == 1:
        plt.loglog( itrs[j], normrs[j], label=fulllabels[j] )

for j in range(len(itrs)):
    if j % 2 == 0:
        plt.loglog( itrs[j], normrs[j], label=fulllabels[j] )

plt.xlabel('iterations')
plt.ylabel('training loss')
plt.grid(color='lightgray',ls='-')
# plt.legend()
# plt.show()

plt.savefig('./figs/LOG_'+mol+'_'+basis+'.pdf',bbox_inches='tight')
plt.close()